In [15]:
import pickle 

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [16]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [17]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))


### 평가

### Baseline (w/o RAG)

In [18]:
from tqdm import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

predictions = []

# 각 질문에 대해 OpenAI 모델을 호출하여 예측 응답을 생성
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI 모델 호출하여 응답 생성
    predictions.append(prediction)  # 생성된 응답을 predictions 리스트에 추가


100%|██████████| 10/10 [01:15<00:00,  7.53s/it]


In [19]:
predictions

['글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:\n\n1. **중앙은행의 통화정책**: 글로벌 금융위기 이후 많은 중앙은행들이 경제 회복을 촉진하기 위해 금리를 낮추고 양적 완화를 시행했습니다. 이러한 정책은 경제 성장을 촉진하고 실업률을 낮추기 위한 목적이었습니다.\n\n2. **저성장 및 저물가**: 선진국을 중심으로 경제 성장률이 낮아지고 물가 상승률이 둔화되었습니다. 저성장과 저물가는 중앙은행이 금리를 낮게 유지하는 주요 요인 중 하나입니다.\n\n3. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 소비와 투자 수요가 감소하고, 이는 경제 성장률을 낮추는 요인으로 작용합니다. 이에 따라 금리도 낮게 유지되는 경향이 있습니다.\n\n4. **글로벌 불확실성**: 무역 분쟁, 지정학적 리스크, 코로나19 팬데믹 등 글로벌 불확실성이 증가하면서 안전자산에 대한 수요가 높아졌습니다. 이는 채권 가격을 상승시키고 금리를 낮추는 결과를 초래했습니다.\n\n5. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추는 요인으로 작용합니다. 물가 상승 압력이 낮아지면 중앙은행은 금리를 낮게 유지할 수 있습니다.\n\n6. **저축 과잉**: 일부 국가에서는 저축률이 높아지면서 자본 공급이 증가하고, 이는 금리를 낮추는 요인으로 작용합니다. 특히 아시아 국가들의 높은 저축률이 글로벌 저금리 현상에 기여하고 있습니다.\n\n이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.',
 '고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높은 위험을 감수하는 투자 상품이나 자산에 집중하는 현상을 의미합니다. 이러한 현상은 여러 가지 요인에 의해 발생할 수 있습니다.\n\n1. **저금리 환경**: 중앙은행의 저금리 정책으로 인해 전통적인 저위험 자산(예: 국채, 예금 등)의 수익률이 낮아지면, 투자자들은 더 높은

In [20]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

# 평가할 샘플 생성
samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,       # 사용자 질문
        response=ai_response,      # AI의 응답
        reference=ground_truth     # 정답
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples)

# Factual Correctness 메트릭 정의
factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

# 평가 수행
results = evaluate(dataset, metrics=[factual_correctness_metric])

# 결과를 데이터프레임으로 변환하여 출력
results_df = results.to_pandas()
print(results_df.head())


Evaluating: 100%|██████████| 10/10 [02:02<00:00, 12.26s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 있습니다. 주요 원인들을 살...   
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행(Investment Banks, IB)들은 다양한...   
4  금융시스템 개혁법은 1999년 11월 12일에 제정되었습니다. 이 법은 공식적으로 ...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.30  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.22  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.18  
3  2000년대 이후 글로벌 IB들은 

In [21]:
print(results)

{'factual_correctness': 0.1220}


In [22]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [23]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **중앙은행의 통화정책**: 글로벌 금융위기 이후 많은 중앙은행들이 경제 회복을 촉진하기 위해 금리를 낮추고 양적 완화를 시행했습니다. 이러한 정책은 경제 성장을 촉진하고 실업률을 낮추기 위한 목적이었습니다.

2. **저성장 및 저물가**: 선진국을 중심으로 경제 성장률이 낮아지고 물가 상승률이 둔화되었습니다. 저성장과 저물가는 중앙은행이 금리를 낮게 유지하는 주요 요인 중 하나입니다.

3. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 소비와 투자 수요가 감소하고, 이는 경제 성장률을 낮추는 요인으로 작용합니다. 이에 따라 금리도 낮게 유지되는 경향이 있습니다.

4. **글로벌 불확실성**: 무역 분쟁, 지정학적 리스크, 코로나19 팬데믹 등 글로벌 불확실성이 증가하면서 안전자산에 대한 수요가 높아졌습니다. 이는 채권 가격을 상승시키고 금리를 낮추는 결과를 초래했습니다.

5. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추는 요인으로 작용합니다. 물가 상승 압력이 낮아지면 중앙은행은 금리를 낮게 유지할 수 있습니다.

6. **저축 과잉**: 일부 국가에서는 저축률이 높아지면서 자본 공급이 증가하고, 이는 금리를 낮추는 요인으로 작용합니다. 특히 아시아 국가들의 높은 저축률이 글로벌 저금리 현상에 기여하고 있습니다.

이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.


In [24]:
import numpy as np
from tqdm import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

# 질문에 대해 가장 유사한 Context를 검색하는 함수
def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]  # 질문의 임베딩 생성
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')  # Context 임베딩 생성

    # 질문과 각 Context 간의 유사도 계산
    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    # 유사도가 가장 높은 Context 인덱스 반환
    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

# 각 질문에 대해 가장 관련 있는 Context를 사용하여 OpenAI 모델에 질문
predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])  # 가장 관련 있는 Context 검색
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # 모델에 질문과 Context로 응답 생성
    predictions.append(prediction)  # 생성된 응답을 predictions 리스트에 추가


100%|██████████| 10/10 [00:55<00:00,  5.56s/it]


In [25]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

# 각 질문, AI 응답, 정답을 사용해 평가할 샘플 생성
samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,       # 사용자 질문
        response=ai_response,      # AI의 응답
        reference=ground_truth     # 정답
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples)

# Factual Correctness 메트릭 설정
factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

# 평가 수행 및 결과 저장
results = evaluate(dataset, metrics=[factual_correctness_metric])

# 결과를 데이터프레임으로 변환하여 출력
results_df = results.to_pandas()
print(results_df.head())


Evaluating: 100%|██████████| 10/10 [01:40<00:00, 10.04s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 ...   
1  고수익-고위험 부문으로의 쏠림현상은 저금리 기조 하에서 투자자들이 원하는 수익률을 ...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행들(IB)은 여러 전략을 추진하였습니다. 주요 전략...   
4  금융시스템 개혁법에 대한 구체적인 제정 시기는 제공된 문맥에서 직접적으로 언급되지 ...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.67  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.67  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.44  
3  2000년대 이후 글로벌 IB들은 

In [26]:
print(results)

{'factual_correctness': 0.5800}


In [27]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이로 인해 글로벌 저금리 현상이 부각되었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태도 저금리 현상을 지속시키는 요인 중 하나입니다. 팬데믹이 진정되더라도 글로벌 저금리 추세는 계속될 가능성이 높습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.


In [28]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
